In [ ]:
import sqlite3

DB_PATH = r"D:\lanzhengpeng\LLMBroker\config.db"  # 你的 SQLite 数据库文件路径

# 厂商数据
providers = [
    ("zhipu", "智谱AI"),
    ("xunfei", "讯飞星火"),
    ("baidu", "百度文心"),
    ("google", "谷歌 Gemini"),
    ("tenxun", "腾讯混元")
]

# 连接数据库
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# 插入厂商数据（去重插入）
for name, display_name in providers:
    cursor.execute(
        "INSERT OR IGNORE INTO model_providers (name, display_name) VALUES (?, ?)",
        (name, display_name)
    )

conn.commit()
conn.close()

print("✅ 厂商表已初始化完成！")


✅ 厂商表已初始化完成！


In [5]:
import sqlite3

DB_PATH = r"D:\lanzhengpeng\LLMBroker\config.db"

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print("数据库表：", cursor.fetchall())

conn.close()


数据库表： [('model_providers',), ('sqlite_sequence',), ('models',), ('model_apis',)]


In [9]:
import json
import sqlite3

DB_PATH = r"D:\lanzhengpeng\LLMBroker\config.db"
JSON_PATH = r"../models/model_name.json"  # 你存模型配置的json文件路径

# 读取json文件
with open(JSON_PATH, 'r', encoding='utf-8') as f:
    model_config = json.load(f)

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

for provider_name, models in model_config.items():
    cursor.execute("SELECT id FROM model_providers WHERE name = ?", (provider_name,))
    row = cursor.fetchone()
    if not row:
        print(f"厂商 {provider_name} 不存在，跳过")
        continue
    provider_id = row[0]

    for model_key, model_id in models.items():
        cursor.execute("""
            INSERT OR IGNORE INTO models (provider_id, model_name, model_id)
            VALUES (?, ?, ?)
            """, (provider_id, model_key, model_id))


conn.commit()
conn.close()

print("模型数据插入完成！")


模型数据插入完成！


In [14]:
import json
import sqlite3

DB_PATH = r"D:\lanzhengpeng\LLMBroker\config.db"
DETAILS_JSON_PATH = r"../models/zhipu.json"  # 详细信息的json路径

# 读取详细信息 JSON
with open(DETAILS_JSON_PATH, 'r', encoding='utf-8') as f:
    model_details_list = json.load(f)

# 建一个字典，key 是 model_id，方便快速查找
model_details_dict = {item['id']: item for item in model_details_list}

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

for model_id, detail in model_details_dict.items():
    description = detail.get('description')
    created = detail.get('created')

    # 更新 models 表中对应 model_id 的 description 和 created
    cursor.execute("""
        UPDATE models
        SET description = ?, created = ?
        WHERE model_id = ?
    """, (description, created, model_id))

conn.commit()
conn.close()

print("模型描述和创建时间更新完成！")


模型描述和创建时间更新完成！


In [16]:
import json
import sqlite3

DB_PATH = r"D:\lanzhengpeng\LLMBroker\config.db"
API_CONFIG_PATH = r"../models/model_config.json"  # 你的API配置JSON文件路径

with open(API_CONFIG_PATH, 'r', encoding='utf-8') as f:
    api_config = json.load(f)

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

for provider_name, models in api_config.items():
    # 查询厂商id
    cursor.execute("SELECT id FROM model_providers WHERE name = ?", (provider_name,))
    row = cursor.fetchone()
    if not row:
        print(f"厂商 {provider_name} 不存在，跳过")
        continue
    provider_id = row[0]

    for model_name, api_info in models.items():
        # 查询模型id
        cursor.execute(
            "SELECT id FROM models WHERE provider_id = ? AND model_name = ?", 
            (provider_id, model_name)
        )
        model_row = cursor.fetchone()
        if not model_row:
            print(f"模型 {model_name} 不存在，跳过")
            continue
        model_pk_id = model_row[0]

        base_url = api_info.get("base_url")
        api_key = api_info.get("api_key")

        cursor.execute("""
            INSERT INTO model_apis (model_id, base_url, api_key)
            VALUES (?, ?, ?)
        """, (model_pk_id, base_url, api_key))

conn.commit()
conn.close()

print("API 配置插入完成！")


API 配置插入完成！
